In [1]:
import numpy as np
from scipy.integrate import solve_ivp

def cr_cs(t, XY, R, Y0):
    """
    Defines the system of differential equations.

    Parameters:
    t (float): Time variable.
    XY (array): Array containing X and Y variables.
    R (array): Coefficient matrix.
    Y0 (array): Initial Y values.

    Returns:
    array: Derivatives of X and Y.
    """
    d0 = 1
    N = R.shape[0]
    X = XY[:N]
    Y = XY[N:]
    
    dXdt = X * ((R @ Y) - d0)
    dYdt = Y0 - Y * ((R.T @ X) + d0)
    
    return np.concatenate([dXdt, dYdt])

def stat_fun_cs(t, XY, R, Y0):
    """
    Event function to detect steady state.

    Parameters:
    t (float): Time variable.
    XY (array): Array containing X and Y variables.
    R (array): Coefficient matrix.
    Y0 (array): Initial Y values.

    Returns:
    float: Value of the event function.
    """
    return np.all(np.abs(cr_cs(t, XY, R, Y0)) - 1e-5 < 0)

stat_fun_cs.terminal = True
stat_fun_cs.direction = 0

def find_ss_cs(R, XY0):
    """
    Finds the steady-state solution of the system.

    Parameters:
    R (array): Coefficient matrix.
    XY0 (array): Initial guess for X and Y.

    Returns:
    ts (array): Time points.
    XYs (array): Solution array containing X and Y over time.
    """
    N, M = R.shape
    X0 = XY0[:N]
    Y0 = XY0[N:]
    
    t0 = 0
    t1 = 1e2

    # First solve
    sol = solve_ivp(lambda t, XY: cr_cs(t, XY, R, Y0), [t0, t1], XY0, method='RK45',
                    rtol=1e-5, atol=1e-6, events=lambda t, XY: stat_fun_cs(t, XY, R, Y0))
    XYs = sol.y.T

    # Trim
    dXs = (XYs[-1, :N] - XYs[-2, :N]) / XYs[-2, :N]
    X0[dXs < -1e-4] = 0

    # Solve again
    sol = solve_ivp(lambda t, XY: cr_cs(t, XY, R, Y0), [t0, t1], np.concatenate([X0, Y0]), method='RK45',
                    rtol=1e-6, atol=1e-7, events=lambda t, XY: stat_fun_cs(t, XY, R, Y0))
    
    return sol.t, sol.y.T

# Example usage


In [2]:
import numpy as np
import scipy.io as sio
import os

# Load the .mat files
data = sio.loadmat('HMP_start_converted.mat')

In [30]:
# Extract necessary data from the loaded .mat files
abundances = data['abundances']
inds_species = data['inds_species'].flatten() - 1
header = data['header']
GCNs = data['GCNs']
img_ids = data['img_ids'].flatten()  # Extract the first column to get the correct shape
keggs = data['keggs'].flatten()  # Extract the first column to get the correct shape

In [31]:
# Select samples
relabus = abundances[inds_species, :]  # Select species of index
inds_samps = (header[4, :] == 'Stool') #& (header[7, :] == '1')  # Identify relevant samples
relabus = relabus[:, inds_samps]  # Select only the relevant samples

# Identify rows with all values <= 1e-6
inds2del = np.all(relabus <= 1e-6, axis=1)  # Identify rows with all values <= 1e-6
relabus = relabus[~inds2del, :]  # Remove these rows from relabus
G = GCNs > 0  # Convert GCNs to boolean
G = G[~inds2del, :]  # Remove the same rows from G
img_ids = img_ids[~inds2del]  # Remove the same indices from img_ids

# Identify columns with coverage < 0.8
coverage = np.sum(relabus[img_ids > 0, :], axis=0)  # Calculate coverage
inds2del = coverage < 0.8  # Identify columns with coverage < 0.8
relabus = relabus[:, ~inds2del]  # Remove these columns from relabus

# Identify columns in G where the sum is 0
inds2del = np.sum(G, axis=0) == 0  # Identify columns in G where the sum is 0
G = G[:, ~inds2del]  # Remove these columns from G
keggs = keggs[~inds2del]  # Remove corresponding elements from keggs



relabus = np.nan_to_num(relabus)  # Replace NaNs with 0
relabus = relabus / np.sum(relabus, axis=0, keepdims=True)  # Normalize relabus



In [32]:
# Get(30, 200) for simulation from relabus

row_sums = np.sum(relabus, axis=1)
sorted_indices = np.argsort(row_sums)[::-1]
relabus = relabus[sorted_indices]
relabus = relabus[:200,:]
inds2del = (np.sum(relabus, axis=0, keepdims=True) ==0).flatten()
relabus = relabus[:,~inds2del]
simulation = relabus[:,:200]
simulation = np.nan_to_num(simulation)
simulation = simulation / np.sum(simulation, axis=0, keepdims=True)
mm  = simulation.shape[1]

In [34]:
mx = np.mean(simulation, axis=1)
n1_orig = (np.array([0.58695652, 0.33423913, 0.45652174, 0.30706522, 0.36684783,
       0.37771739, 0.37771739, 0.69836957, 0.71467391, 0.6548913])*30).astype(int)
dm_origs = np.array([ 0.78799916,  0.49779242,  0.57094609, -0.29606619,  0.64892559,
       -0.47545641, -0.47545641,  0.75077074,  0.71582043,  0.8309626])
N = 200
M = 10
def ConvertToG(G, relabus):
    temp = np.dot(relabus.T, G).T
    return temp / np.sum(temp, axis=0, keepdims=True)

#一种相似度计算方法
def CalculateBC(xthis, ythis):
    return 1 - np.sum(np.minimum(xthis, ythis))

# 计算每列与均值的相似度度量
def CalculateDs(xs):
    mm = xs.shape[1]
    xm = np.mean(xs, axis=1, keepdims=True)
    ds = np.zeros(mm)
    for iii in range(mm):
        ds[iii] = CalculateBC(xs[:, iii], xm.flatten())
    return ds

def shuffle_matrix_rows(matrix):
    # 对矩阵进行复制，防止原矩阵被修改
    shuffled_matrix = np.copy(matrix)
    # 对于矩阵中的每一行进行随机排列
    for row in shuffled_matrix:
        np.random.shuffle(row)
    return shuffled_matrix


def rearrange_rows(matrix):
    """
    Rearranges each row of the matrix while keeping the column sums nearly unchanged.

    Parameters:
    matrix (array): Input 2D array.

    Returns:
    array: Rearranged matrix.
    """
    # Calculate the target column sums
    target_col_sums = np.sum(matrix, axis=0)
    
    # Initialize the rearranged matrix
    rearranged_matrix = np.zeros_like(matrix)
    
    # Iterate over each row
    for i in range(matrix.shape[0]):
        # Get the current row
        row = matrix[i, :]
        
        # Sort the row in ascending order
        sorted_row = np.sort(row)
        
        # Calculate the current column sums
        current_col_sums = np.sum(rearranged_matrix, axis=0)
        
        # Calculate the differences between the target column sums and the current column sums
        col_diff = target_col_sums - current_col_sums
        
        # Rearrange the sorted row elements to minimize the difference with the target column sums
        indices = np.argsort(col_diff)
        rearranged_matrix[i, indices] = sorted_row
    
    return rearranged_matrix

def svd_rearrange(matrix):
    """
    Rearranges the rows of a matrix using SVD and shuffling of the U and V matrices.

    Parameters:
    matrix (array): Input 2D array.

    Returns:
    array: Rearranged matrix.
    """
    # Perform SVD decomposition
    U, Sigma, VT = np.linalg.svd(matrix, full_matrices=False)
    
    # Shuffle rows of U and columns of V
    np.random.shuffle(U)
    np.random.shuffle(VT.T)  # Shuffle the columns of V (or equivalently rows of V^T)
    
    # Reconstruct the matrix
    rearranged_matrix = np.dot(U, np.dot(np.diag(Sigma), VT))
    
    return rearranged_matrix


In [35]:
import numpy as np

def generate_matrix(rows, cols):
    """
    Generates a random 0-1 matrix where the number of 1s in each column
    decreases from left to right and the proportion of 1s is between 0.8 and 0.2.

    Parameters:
    rows (int): Number of rows in the matrix.
    cols (int): Number of columns in the matrix.

    Returns:
    array: A 2D array representing the random 0-1 matrix.
    """
    # Calculate the proportion of 1s for each column
    proportions = np.linspace(0.8, 0.2, cols)
    
    # Initialize the matrix with zeros
    matrix = np.zeros((rows, cols), dtype=int)
    
    for col in range(cols):
        # Determine the number of 1s in this column
        num_ones = int(proportions[col] * rows)
        
        # Generate random indices to place the 1s
        ones_indices = np.random.choice(rows, num_ones, replace=False)
        
        # Set the selected indices to 1
        matrix[ones_indices, col] = 1
    
    return matrix


# Example usage
rows = 200  # Number of rows in the matrix
cols = 10   # Number of columns in the matrix
G_r = generate_matrix(rows, cols)


In [39]:
R = G_r
XY0 = np.concatenate((simulation[:,0],np.sum(G_r, axis=0).T))
ts, XYs = find_ss_cs(R, XY0)


C:\Users\lenovo\AppData\Local\Temp\ipykernel_37916\3970736644.py:70: RuntimeWarning: invalid value encountered in divide
  dXs = (XYs[-1, :N] - XYs[-2, :N]) / XYs[-2, :N]


In [42]:
XYs[:, :N] /= np.sum(XYs[:, :N], axis=1, keepdims=True)
XYs[:,N+1:]

array([[1.46000000e+02, 1.33000000e+02, 1.20000000e+02, ...,
        6.60000000e+01, 5.30000000e+01, 4.00000000e+01],
       [1.45991232e+02, 1.32996600e+02, 1.19992794e+02, ...,
        6.59975657e+01, 5.29969439e+01, 3.99985247e+01],
       [1.45976608e+02, 1.32991004e+02, 1.19980774e+02, ...,
        6.59934323e+01, 5.29918751e+01, 3.99960196e+01],
       ...,
       [1.46478595e-01, 1.63584939e-01, 1.20392203e-01, ...,
        6.73706403e-02, 2.62654181e-01, 4.08306911e-02],
       [1.46478817e-01, 1.63585029e-01, 1.20392007e-01, ...,
        6.73706043e-02, 2.62654282e-01, 4.08306693e-02],
       [1.46478086e-01, 1.63585074e-01, 1.20392649e-01, ...,
        6.73705864e-02, 2.62654333e-01, 4.08306584e-02]])